# Optional Homework (Due 11/33/2019, 11:59pm)

## Automatic Differentiation

**AM 207: Advanced Scientific Computing**<br>
**Instructor: Weiwei Pan**<br>
**Fall 2019**

**Name: Ian Weaver**

**Students collaborators:**

## Problem Description

1. Implement forward mode auto-differentiation, for functions composed of 2 binary arithmetic operations (+, -) and four elementary functions (constant, power n, sin, ln).


2. Implement reverse mode auto-differentiation (tweak or add to the code provided in Lecture #16), for functions composed of 2 binary arithmetic operations (+, -) and four elementary functions (constant, power n, sin, ln).


3. Verify that your implementation computes derivatives correctly by comparing the derivative your implementation of auto-diff (both modes) computes versus the derivatives you compute by hand (or have Wolfram Alpha compute).


4. Implement automatic Hessian computation by composing forward mode and reverse mode differentiation.

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import scipy as sp
%matplotlib inline

In [2]:
# Plot configs
fig_wide = (11, 4)
%config InlineBackend.figure_format = "retina"
%matplotlib inline
sns.set(style="darkgrid", palette="colorblind", color_codes=True)

## 1.1 Forward Mode Automatic Differentiation

### Background
This is a fantastic method that can simultaneously compute a function and its 
derivative to machine precision! This can be done by introducing the dual number
$\epsilon^2=0$, where $\epsilon \ne 0$. If we transform some arbitrary function $f(x)$ 
to $f(x+\epsilon)$ and Taylor expand it, we have: 

$$
    f(x+\epsilon) = f(x) + \epsilon f'(x) + O\left(\epsilon^2\right)\quad.
$$

Applying the definition $\epsilon \equiv 0$, all second order and higher terms in
$\epsilon$ vanish and we are left with 
$f(x+\epsilon) = f(x) + \epsilon f'(x)$, where the dual part, $f'(x)$ of this transformed
function is the derivative of our original function $f(x)$. If we adhere
to the new system of math introduced by dual numbers, we are able to compute
derivatives of functions exactly! 

For example, multiplying two dual numbers $z_1 = a_r + \epsilon a_d$ and 
$z_2 = b_r + \epsilon b_d$ would behave like:

\begin{align}
    z_1 \times z_2 &= (a_r + \epsilon a_d) \times (b_r + \epsilon b_d)
    = a_rb_r + \epsilon(a_rb_d + a_db_r) + \epsilon^2 a_db_d \\
    &= \boxed{a_rb_r + \epsilon(a_rb_d + a_db_r)}\quad.
\end{align}

When the derivative of an elementary function is already known, we could also just do something like this:

$$
    \ln(x) \longrightarrow \ln(x_r + \epsilon x_d)
    = \ln(x_r) + \epsilon \frac{x_d}{x_r} \quad,
$$

If we set $x_d = 1$, we automatically get the derivative of $x_r$. Operations like the ones shown above can be redefined via operator overloading. 

This method is also readily extended to multivariable functions with
the introduction of **dual number basis vectors**
$\pmb p_i = i + \epsilon_i 1$, where $i$ takes on any of the
components of $\pmb X_n$. For example, the multivariable function
$f(x, y) = xy$ would transform like:

\begin{align}
    \require{cancel}
    x \quad\longrightarrow\quad& \pmb p_x = x + \epsilon_x + \epsilon_y\ 0 \\
    y \quad\longrightarrow\quad& \pmb p_y = y + \epsilon_x\ 0 + \epsilon_y \\
    f(x, y) \quad\longrightarrow\quad& f(\pmb p) = (x + \epsilon_x + \epsilon_y\ 0) 
    \times (y + \epsilon_x\ 0 + \epsilon_y) \\
    &= xy + \epsilon_y x + \epsilon_x y + 
    \cancel{\epsilon_x\epsilon_y} \\
    &= xy + \epsilon_x y + \epsilon_y x \quad,
\end{align}

where we now have:

\begin{align}
    \newcommand{\pd}[2]{\frac{\partial#1}{\partial#2}}
    f(x+\epsilon_x, y+\epsilon_y) 
    &= f(x, y) + \epsilon_x\pd{f}{x} + \epsilon_y\pd{f}{y} 
    = f(x, y) + \epsilon \left[\pd{f}{x},\ \pd{f}{y}\right] \\
    &= f(x, y) + \epsilon \nabla f(x, y)\quad.
\end{align}

### Implementation
The implementation of the overloaded multiplication operator along with `+`, `-`, `pow n`, `sin`, `ln`, are included in the `Dual` number class below:

In [92]:
# adapted from a pacakge I worked on
# https://spacejam.readthedocs.io/en/latest/index.html
def _toDual(obj):
        if isinstance(obj, (int, float)):
            return Dual(obj, 0)
        elif isinstance(obj, Dual):
            return obj
        else:
            raise Exception(f"Type {type(obj)} not supported")
            
class Dual():
    """ Creates dual numbers and defines dual number math.
    A real number `a` is taken in and its dual counterpart `a + eps [1.00]` is
    returned to facilitate automatic differentiation in `ForwardDiff` .
    
    Notes
    -----
    The dual part can optionally be returned as a "dual basis vector"
    [0 1 0] if the user function `f` is multivariable and the partial
    derivative $\partial f / \partial x_2$ is desired, for example.
    
    Attributes
    ----------
    r : float
        real part of `Dual` .
    d : numpy.ndarray
        dual part of `Dual` .
    """
    def __init__(self, real, dual):
        """
        Parameters
        ----------
        real : int/float
            real part of `Dual` .
        dual : float
            dual part of `Dual` .
        """
        self.r = real
        self.d = dual
        self.children = []
    
    def __add__(self, other):
        """ Returns the addition of self and other
        
        Parameters
        ----------
        self: Dual object
        other: Dual object, float, or int
        
        Returns
        ------- 
        z: Dual object that is the sum of self and other
        """
        other = _toDual(other)
        real = self.r + other.r
        dual = self.d + other.d

        z = Dual(real, dual)
        self.children.append((1.0, z))
        other.children.append((1.0, z))
        return z
    
    __radd__ = __add__ # addition commutes
    
    def __sub__(self, other):
        """ Returns the subtraction of self and other
        
        Parameters
        ----------
        self: Dual object
        other: Dual object, float, or int
        
        Returns
        ------- 
        z: Dual object
           difference of self and other
           
        NOTES
        ----- 
        Subtraction does not commute in general. 
        A specialized __rsub__ is required.
        
        Examples
        """
        other = _toDual(other)
        real = self.r - other.r
        dual = self.d - other.d
        
        z = Dual(real, dual)
        self.children.append((1.0, z))
        other.children.append((-1.0, z))
        return z
    
    def __rsub__(self, other):
        """ Returns other - self
        
        Parameters
        ----------
        self: Dual object
        other: Dual object, float, or int
        
        Returns
        ------- 
        z: Dual object
           difference of other and self
        """
        real = other - self.r
        dual = -self.d
        
        z = Dual(real, dual)
        self.children.append((-1.0, z))
        other.children.append((1.0, z))
        return z
    
    def __mul__(self, other):
        """ Returns the product of self and other
        
        Parameters
        ----------
        self: Dual object
        other: Dual object, float, or int
        
        Returns
        ------- 
        z: Dual object that is the product of self and other
        """
        other = _toDual(other)
        real = self.r*other.r 
        dual = self.r*other.d + self.d*other.r
        
        z = Dual(real, dual)
        self.children.append((other.r, z))
        other.children.append((self.r, z))
        return z

    __rmul__ = __mul__ # multiplication commutes
    
    def __pow__(self, n):
        """ Performs (self.r + eps self.d)**n
        Parameters
        ----------
        self: Dual object
        n: int or float
        
        Returns
        ------- 
        z: Dual object that is self raised to the other power
        """
        real = self.r**n
        dual = self.r**(n - 1)*self.d*n
        
        z = Dual(real, dual)
        self.children.append((n*self.r, z))
        return z


    # overload numpy functions
    def log(self):
        # Note: base can be changed with log(x) / log(base)
        real = np.log(self.r)
        dual = self.d / self.r
        z = Dual(real, dual)
        self.children.append((1./self.r, z))
        return z

    def sin(self):
        real = np.sin(self.r)
        dual = self.d*np.cos(self.r)
        z = Dual(real, dual)
        self.children.append((np.cos(self.r), z))
        return z
    
    # format printed output
    def __repr__(self):
        s = f"{self.r:} + eps {self.d}"
        return s
        
class ForwardDiff():
    """ Performs automatic differentiation (AD) on functions input by user.
    AD if performed by transforming `f(x1, x2, ...)` to `f(p_x1, p_x2, ...)`,
    where `p_xi` is returned from `Dual` . 
    The final result is then returned as a `numpy.ndarray`
    
    Attributes
    ----------
    r : numpy.ndarray
        User defined function(s) `F` evaluated at `p`.
    d : numpy.ndarray
        Corresponding derivative, gradient, or Jacobian of user defined
        functions(s).
    """

    def __init__(self, func, p):
        """ 
        Parameters
        ----------
        func : numpy.ndarray
            user defined function(s).
        p : int or numpy.ndarray
            user defined point(s) to evaluate derivative at
        """
        result = self._ad(func, p) 
        self.r = result.r
        self.d = result.d

    def _ad(self, func, p):
        """ Internally computes `func(p)` and its derivative(s).
        
        Notes
        -----
        `_ad` returns a nested 1D `numpy.ndarray` to be formatted internally
        accordingly in `ForwardDiff.__init__`.
        
        Parameters
        ----------
        func : numpy.ndarray
            function(s) specified by user.
        p : numpy.ndarray
            point(s) specified by user.
        """
        # build p_i = p + eps i
        number_of_inputs = p.size
        duals = np.identity(number_of_inputs)
        ps = [Dual(p[i], duals[i, :]) for i in range(number_of_inputs)]

        # perform AD with specified function
        result = func(*ps) 
        return result

## 1.2 Reverse Mode Automatic Differentiation

## Background

See https://rufflewind.com/2016-12-30/reverse-mode-automatic-differentiation and Lecture 17.

The main takeaway is that in forward mode automatic differentiation, like in 1.1, we are doing something like this (via the chain rule):

``` python
    # computes z = x*y + sin(x) and its derivative wrt. t
    # where,
    # x = ?
    # y = ?
    # a = x*y
    # b = sin(x)
    # z = a + b = x*y + sin(x)
    x  = ?
    dxdt = ?

    y  = ?
    dydt = ?

    a  = x * y
    dadt = y * dxdt + x * dydt

    b  = sin(x)
    dbdt = cos(x) * dxdt

    z  = a + b
    dzdt = da + db
```

where the notation `dwdt` means derivative of `w` wrt. some input variable `t`. This is nice because it lends itself well to implementation using dual numbers, but it has a major disadvantage: scaling. Saw we want to calculate both `dzdx` and `dzdy` for some `x` and `y`. We would first have to seed `dxdt = 1` and `dydt = 0` run the code block above to get `dzdx`, then re-seed with `dxdt = 0` and `dydt = 1`, then re-run the block a second time to get `dzdy` . This way of computing things scales like $O(n)$ then, for $n$ inputs, which is bad for [applications](https://en.wikipedia.org/wiki/Mathematical_optimization) where $n$ can be very large.

The solution is to take advantage of the symmetry of the chain rule to invert the roles of the input and output. In other words: "instead of asking what input variables a given output variable depends on, we have to ask what output variables a given input variable can affect". A forward and reverse pass as described in lecture will then allow us to compute the necessary gradients!

## Implementation

In [12]:
def _toVar(obj):
        if isinstance(obj, (int, float)):
            return Var(obj)
        elif isinstance(obj, Var):
            return obj
        else:
            raise Exception(f"Type {type(obj)} not supported")
            
'''small example of reverse mode autodiff as implemented in https://github.com/Rufflewind/revad/'''
class Var:
    def __init__(self, value):
        self.value = value
        self.children = []
        self.grad_value = None

    def grad(self):
        # propogate derivatives
        if self.grad_value is None:
            self.grad_value = sum(weight * var.grad()
                                  for weight, var in self.children)
        return self.grad_value

    def __add__(self, other):
        other = _toVar(other)
        z = Var(self.value + other.value)
        self.children.append((1.0, z))
        other.children.append((1.0, z))
        return z
    __radd__ = __add__
    
    def __sub__(self, other):
        other = _toVar(other)
        z = Var(self.value - other.value)
        self.children.append((1.0, z))
        other.children.append((-1.0, z))
        return z
    __rsub__ = __sub__
    
    def __mul__(self, other):
        other = _toVar(other)
        z = Var(self.value * other.value)
        self.children.append((other.value, z))
        other.children.append((self.value, z))
        return z
    __rmul__ = __mul__

    def __pow__(self, n):
        z = Var(self.value**n)
        self.children.append((n*self.value, z))
        return z

    def sin(self):
        z = Var(np.sin(self.value))
        self.children.append((np.cos(self.value), z))
        return z

    def log(self):
        z = Var(np.log(self.value))
        self.children.append((1./self.value, z))
        return z

## 1.3 Verify implementations

For the arbitrary function `f` below using the elementary operations 
`+`, `-`, `pow n`, `sin`, and `log` we get the following results for the forward and reverse modes:

In [93]:
def f(x, y):
    return 4 + x - np.sin(3*x*np.log(y**2)) + np.log(x**2)

p = np.array([0.5, 4.2]) # location to evaluate f(p) at

# forward mode 
ad = ForwardDiff(f, p)
print("forward")
print("-------")
print(f"p = {p}")
print(f"f(p) = {ad.r}")
print(f"grad[f(p)] = {ad.d}")

# reverse mode
x = Var(p[0])
y = Var(p[1])
z = f(x, y)
z.grad_value = 1 # initial seed
print("\nreverse")
print("-------")
grad = np.array([x.grad(), y.grad()])
print(f"p = {p}")
print(f"f(p) = {z.value}")
print(f"grad[f(p)] = {grad}")

forward
-------
p = [0.5 4.2]
f(p) = 4.031964551713947
grad[f(p)] = [8.40959306 0.28284323]

reverse
-------
p = [0.5 4.2]
f(p) = 4.031964551713947
grad[f(p)] = [8.40959306 0.28284323]


Here, `p` is the starting vector that we evaluate `f(p)` at, and `grad[f(p)]` is the resulting gradient. Hopefully they match for both modes!

## 1.4 Hessian 

The idea here is to compose our forward and reverse modes to compute the Hessian matrix (H), which has the following form:
$\newcommand{\pd}[1]{\frac{\partial f}{\partial{#1}}}$
$\newcommand{\ppd}[1]{\frac{\partial^2 f}{\partial{#1}^2}}$
$\newcommand{\mpd}[2]{\frac{\partial^2 f}{\partial{#1}\partial{#2}}}$
$$
    H = 
    \begin{bmatrix}
    \ppd{x_1}      & \cdots & \mpd{x_1}{x_n} \\
    \vdots         & \ddots & \vdots \\
    \mpd{x_n}{x_1} & \cdots & \ppd{x_n}
    \end{bmatrix}
    =
    \begin{bmatrix}
    \nabla f_{x_1} \cdots \nabla f_{x_n}
    \end{bmatrix}
    \quad,
$$

where $f_{x_i} = \pd{x_i}$. Each column of the Hessian is a gradient, which is perfect for automatic differentiation. The reverse-on-forward method first computes the gradient vector products with the forward mode implementation. We then apply the reverse mode implementation to the result to build the Hessian. The second step needs the intermediate gradient values computed in the first step, so I added an analogous `children` field to the `Dual` class to store each node in the forward mode computation graph. Using this syntax instead of the `ForwardDiff` helper class gives me something like this:

In [124]:
def f(x, y):
    return 4 + x - np.sin(3*x*np.log(y**2)) + np.log(x**2)

p = np.array([0.5, 4.2]) # location to evaluate f(p) at


# forward step 
x = Dual(p[0], np.array([1.0, 0.0]))
y = Dual(p[1], np.array([0.0, 1.0]))
z = f(x, y)
print(f"x children: {x.children}")
print(f"y children: {y.children}")

x children: [(1.0, 4.5 + eps [1. 0.]), (3, 1.5 + eps [3. 0.]), (1.0, 0.25 + eps [1. 0.])]
y children: [(8.4, 17.64 + eps [0.  8.4])]


With $f(p)$ and $f'(p)$ stored in the real and dual part of `z`, respectively:

In [126]:
print(z)

4.031964551713947 + eps [8.40959306 0.28284323]


We would then need to back propagate on the forward mode computation graph by composing the reverse mode with the forward mode implementation to get the second order partials that make up the Hessian. There is a very interesting discussion about it [here](https://discourse.julialang.org/t/is-there-an-efficient-way-to-compute-the-hessian-of-a-nn/26971/2) which implements this in `SparseDiffTools.jl` and [here](https://jax.readthedocs.io/en/latest/notebooks/autodiff_cookbook.html#Hessian-vector-products-with-grad-of-grad) using `Jax`.